# Connect to Weights and Biases Servers for Logging and Import Modules

In [1]:
#!pip install wandb 
import wandb
wandb.login() # Log in to your W&B account

from waves.misc import flatten
from waves.load_data import load_json_file_from_path
from waves.load_data import get_ml_dataset_paths
from waves.ml.acoustic_emission_dataset import AcousticEmissionDataset
from waves.ml.model_architectures import NeuralNetwork_01, NeuralNetwork_02, NeuralNetwork_03

import os
import random
import numpy as np
import torch
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report
from torch.utils.data import Dataset
from torch import tensor
import torch.nn as nn
from tqdm.notebook import tqdm
import re, json, requests

# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
torch.manual_seed(hash("by removing stochasticity") % 2**32 - 1)
torch.cuda.manual_seed_all(hash("so runs are repeatable") % 2**32 - 1)

# Device configuration
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

wandb: Currently logged in as: ntulshibagwale (ae-ml_ucsb). Use `wandb login --relogin` to force relogin


# Define Machine Learning Pipeline and Utility Functions

In [2]:
def model_pipeline(config=None): 
    
    # tell wandb to get started
    with wandb.init(project=None, config=config):

        # access all HPs through wandb.config, so logging matches execution!
        config = wandb.config

        print("Hyper-parameters:")
        for key,value in config.items():
            print(key, ':', value)
        print("")

        # make the model, get train and valid data, and optimization problem
        model, train_loader, valid_loader, loss_func, optimizer = make(config)
        print(model)

        # Training!
        print('-----------------------------------------------------------------')
        print("Begin model training...\n")
        train(model, train_loader, loss_func, optimizer, config)
        print("Training completed.\n")
        print('-----------------------------------------------------------------')

        print("Begin model evaluation...\n")
        print("\nEvaluate results on validation data...\n")
        # Evaluate on validation dataset
        evaluate(model, valid_loader, config, valid = True)

        # Save model to w&b 
        print('Saving model to run directory...\n')
        torch.save(model.state_dict(), wandb.run.dir+'\model.pth')

    
    print('\nModel was trained and evaluated, and results were logged!')
    
    return model

def make(config):
    
    # Get the train and validation data from selected ml folder
    train, valid, feature_dim, num_classes = get_data(config) 
    wandb.log({"feature_dim": feature_dim})
    wandb.log({"num_classes": num_classes})

    # Log a plot illustrating ratio of test to train data for each class
    y_valid = torch.argmax(valid[:][1], dim=1)
    y_train = torch.argmax(train[:][1], dim=1)
    wandb.sklearn.plot_class_proportions(y_train,y_valid, config.location)
        
    # Create data loaders
    train_loader = make_loader(train, batch_size=config.batch_size)
    valid_loader = make_loader(valid, batch_size=config.batch_size)
  
    # Make the model according to configuration and available models
    if config.architecture == 1: # 1 layer
        model = NeuralNetwork_01(feature_dim, num_classes, config.hidden_units).to(device)
    if config.architecture == 2: # 2 layer
        model = NeuralNetwork_02(feature_dim, num_classes, config.hidden_units).to(device)
    if config.architecture == 3: # 3 layer
        model = NeuralNetwork_03(feature_dim, num_classes, config.hidden_units).to(device)
        
    # Make the loss and optimizer
    loss_func = nn.MSELoss()
    optimizer = torch.optim.Adam(
        model.parameters(), lr=config.learning_rate)
    
    # Look at shapes of batches
    for ex_idx, (batch_x,batch_y) in enumerate(train_loader):
        print(f"Shape of x batch is: {batch_x.shape}")
        print(f"Datatype of x batch is: {batch_x.dtype}\n")
        print(f"Shape of y batch is: {batch_y.shape}")
        print(f"Datatype of y batch is: {batch_y.dtype}\n")
        break

    return model, train_loader, valid_loader, loss_func, optimizer

def get_data(config):
    
    print('Getting train and validation test sets...')
    train = AcousticEmissionDataset(config.train_path,
                                         config.sig_len,
                                         config.dt,
                                         config.low_pass,
                                         config.high_pass,
                                         config.fft_units,
                                         config.num_bins,
                                         config.feature)
    
    valid = AcousticEmissionDataset(config.valid_path,
                                     config.sig_len,
                                     config.dt,
                                     config.low_pass,
                                     config.high_pass,
                                     config.fft_units,
                                     config.num_bins,
                                     config.feature)
    
    example_x, example_y = train[0] # to determine feature dim

    feature_dim = example_x.shape[0] # for model creation input dim
    num_classes = example_y.shape[0] # for model creation output dim
    
    print(f'\nfeature_dim : {feature_dim}')
    print(f'num_classes : {num_classes}\n')

    return train, valid, feature_dim, num_classes

def make_loader(dataset, batch_size):
    loader = torch.utils.data.DataLoader(dataset=dataset,
                                         batch_size=batch_size, 
                                         shuffle=True,
                                         pin_memory=True, num_workers=2)
    return loader

def train(model, loader, loss_func, optimizer, config):
    
    # Tell wandb to watch what the model gets up to: gradients, weights, etc
    wandb.watch(model, loss_func, log="all", log_freq=20)

    # Run training and track with wandb
    total_batches = len(loader) * config.epochs
    example_ct = 0  # number of examples seen
    batch_ct = 0
    for epoch in tqdm(range(config.epochs)):
        for _, (waves, label) in enumerate(loader):

            loss = train_batch(waves, label, model, optimizer, loss_func)
            example_ct +=  len(waves)
            batch_ct += 1

            # Report metrics every 25th batch
            if ((batch_ct + 1) % 25) == 0:
                train_log(loss, example_ct, epoch)

def train_batch(waves, label, model, optimizer, loss_func):
    
    # Use gpu
    waves, label = waves.to(device), label.to(device)

    # Forward pass ➡
    outputs = model(waves)
    loss = loss_func(outputs, label)
    
    # Backward pass ⬅
    optimizer.zero_grad()
    loss.backward()

    # Step with optimizer
    optimizer.step()

    return loss

def train_log(loss, example_ct, epoch):
    # Where the magic happens
    wandb.log({"epoch": epoch, "loss": loss}, step=example_ct)
    print(f"Loss after " + str(example_ct).zfill(5) + f" examples: {loss:.3f}")

def evaluate(model, loader, config, test = False, valid = True): # would work with test or validation dataset
    model.eval()

    # Run the model on the provided examples
    with torch.no_grad():
        
        eval_predicted = []
        eval_label = []
        correct, total = 0, 0
        i = 0
        
        for waves, label in loader:
            waves, label = waves.to(device), label.to(device)
            outputs = model(waves)
            _, predicted = torch.max(outputs, 1)
            _, label = torch.max(label, 1)
            total += label.size(0)
            correct += (predicted == label).sum().item()
            eval_predicted.append(predicted) # ex. [0,0,1]
            eval_label.append(label)

        print(f"Accuracy of the model on the {total} " +
              f"evaluation waves: {100 * correct / total}%")
        
    print("")
    if valid:
        print("Validation Set Metrics:")
    if test:
        print("Test Set Metrics:")
        
    # Obnoxious coding to get into some correct format
    eval_predicted = [t.cpu().numpy() for t in eval_predicted]
    eval_label = [t.cpu().numpy() for t in eval_label]
    
    eval_label = flatten(eval_label) # from ae functions import
    eval_predicted = flatten(eval_predicted)
    
    print(classification_report(eval_predicted,eval_label,
                                target_names=config.location))
    
    metrics = classification_report(eval_predicted,eval_label,
                                target_names=config.location,
                                    output_dict=True)
    wandb.log(metrics)
    
    if valid:
        wandb.log({"valid_accuracy": correct / total})
        wandb.log({"valid_label": eval_label,"valid_predicted": eval_predicted})
        print('\nModel succesfully evaluated using validation set! \n')
    if test:
        wandb.log({"test_accuracy": correct / total})
        wandb.log({"test_label": eval_label,"test_predicted": eval_predicted})
        print('\nModel succesfully evaluated using test set! \n')


## User selects folder to use containing training, validation and test set 

In [3]:
# User selects ml dataset to use for pipeline
train_path,valid_path,test_path = get_ml_dataset_paths()

C:/Users/tul72/Box/aeml/ml_experiments/ml_01/20220711_ml_01_train.json
C:/Users/tul72/Box/aeml/ml_experiments/ml_01/20220711_ml_01_valid.json
C:/Users/tul72/Box/aeml/ml_experiments/ml_01/20220711_ml_01_test.json


## Input HyperParameter Sweep and Begin Sweep while tracking Experiment Results

In [7]:
sweep_config = {
  "name" : "sweep2",
  "method" : "bayes",
  "metric" : {
    'goal': 'maximize',
    'name': 'valid_accuracy'
  },
  "parameters" : {
    "epochs" : {
      "min": 1,
      "max": 10
    },
    "learning_rate" :{
      "min": 0.0001,
      "max": 0.01
    },
    "batch_size" : {
        "min" : 10,
        "max" : 100
    },
    "feature" : {
        "values" : ['fft','waveform']
    },
    "architecture" : {
        "values" : [1,2,3] # 1 layer, 2 layer, 3 layer
    },
    "hidden_units" : {
        "min" : 20,
        "max" : 100
    },
    "train_path" : {
        "value" : train_path
    },
    "valid_path" : {
        "value" : valid_path
    },
    "test_path" : {
        "value" : test_path
    },
    "sig_len" : {
        "values" : [1024,4096]
    },
    "dt" : {
        "value" : 10**-7
    },
    "low_pass" : {
        "value" : 0
    },
    "high_pass" : {
        "value" : 100*10**4
    },
    "fft_units" : {
        "value" : 1000
    },
    "num_bins" : {
        "value" : 26
    },
    "location" : {
        "value" : ['front', 'tope', 'backcorner']
    }
  }
}

sweep_id = wandb.sweep(sweep_config,project="robustness")
wandb.agent(sweep_id, function=model_pipeline, count=20)

Create sweep with ID: u6pl1gkb
Sweep URL: https://wandb.ai/ae-ml_ucsb/testing_pipeline/sweeps/u6pl1gkb


wandb: Agent Starting Run: q2j5uy9l with config:
wandb: 	architecture: 2
wandb: 	batch_size: 55
wandb: 	dt: 1e-07
wandb: 	epochs: 10
wandb: 	feature: waveform
wandb: 	fft_units: 1000
wandb: 	hidden_units: 41
wandb: 	high_pass: 1000000
wandb: 	learning_rate: 0.004317058257613321
wandb: 	location: ['front', 'tope', 'backcorner']
wandb: 	low_pass: 0
wandb: 	num_bins: 26
wandb: 	sig_len: 4096
wandb: 	test_path: C:/Users/tul72/Box/aeml/ml_experiments/ml_01/20220711_ml_01_test.json
wandb: 	train_path: C:/Users/tul72/Box/aeml/ml_experiments/ml_01/20220711_ml_01_train.json
wandb: 	valid_path: C:/Users/tul72/Box/aeml/ml_experiments/ml_01/20220711_ml_01_valid.json


Hyper-parameters:
architecture : 2
batch_size : 55
dt : 1e-07
epochs : 10
feature : waveform
fft_units : 1000
hidden_units : 41
high_pass : 1000000
learning_rate : 0.004317058257613321
location : ['front', 'tope', 'backcorner']
low_pass : 0
num_bins : 26
sig_len : 4096
test_path : C:/Users/tul72/Box/aeml/ml_experiments/ml_01/20220711_ml_01_test.json
train_path : C:/Users/tul72/Box/aeml/ml_experiments/ml_01/20220711_ml_01_train.json
valid_path : C:/Users/tul72/Box/aeml/ml_experiments/ml_01/20220711_ml_01_valid.json

Getting train and validation test sets...
Loading in Dataset from C:/Users/tul72/Box/aeml/ml_experiments/ml_01/20220711_ml_01_train.json
Successfully loaded in.
Shape of waves is: torch.Size([1152, 4096])
Datatype of waves is: torch.float32
waves requires grad: False
Shape of label is: torch.Size([1152])
Datatype of label is: torch.int64
Label requires grad: False
Ex: label[0] = 0
label_one_hot is the one hot encoding of label for source.  For ex: [1 0 0] is a front occuring

C:\Users\tul72\Box\aeml\waves\ml\acoustic_emission_dataset.py:110: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  x = tensor(x,dtype=torch.float32,requires_grad=False)


Shape of x batch is: torch.Size([55, 4096])
Datatype of x batch is: torch.float32

Shape of y batch is: torch.Size([55, 3])
Datatype of y batch is: torch.float32

NeuralNetwork_02(
  (layers): Sequential(
    (0): Linear(in_features=4096, out_features=41, bias=True)
    (1): ReLU()
    (2): Linear(in_features=41, out_features=41, bias=True)
    (3): ReLU()
    (4): Linear(in_features=41, out_features=3, bias=True)
  )
)
-----------------------------------------------------------------
Begin model training...



...

Shape of x batch is: torch.Size([30, 4096])
Datatype of x batch is: torch.float32

Shape of y batch is: torch.Size([30, 3])
Datatype of y batch is: torch.float32

NeuralNetwork_02(
  (layers): Sequential(
    (0): Linear(in_features=4096, out_features=55, bias=True)
    (1): ReLU()
    (2): Linear(in_features=55, out_features=55, bias=True)
    (3): ReLU()
    (4): Linear(in_features=55, out_features=3, bias=True)
  )
)
-----------------------------------------------------------------
Begin model training...



  0%|          | 0/8 [00:00<?, ?it/s]

Loss after 00720 examples: 0.100
Loss after 01452 examples: 0.028
Loss after 02202 examples: 0.043
Loss after 02934 examples: 0.048
Loss after 03666 examples: 0.017
Loss after 04416 examples: 0.021
Loss after 05148 examples: 0.012
Loss after 05880 examples: 0.007
Loss after 06630 examples: 0.013
Loss after 07362 examples: 0.009
Loss after 08094 examples: 0.028
Loss after 08844 examples: 0.009
Training completed.

-----------------------------------------------------------------
Begin model evaluation...


Evaluate results on validation data...

Accuracy of the model on the 460 evaluation waves: 97.6086956521739%

Validation Set Metrics:
              precision    recall  f1-score   support

       front       1.00      0.99      1.00       167
        tope       0.99      0.94      0.96       144
  backcorner       0.94      0.99      0.97       149

    accuracy                           0.98       460
   macro avg       0.98      0.97      0.98       460
weighted avg       0.98      

accuracy,▁
epoch,▁▂▂▃▄▄▅▆▆▇██
feature_dim,▁
loss,█▃▄▄▂▂▁▁▁▁▃▁
num_classes,▁
valid_accuracy,▁
accuracy,0.97609
epoch,7
feature_dim,4096
loss,0.00899
num_classes,3



Model was trained and evaluated, and results were logged!
